In [32]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

from IPython.display import IFrame

In [33]:
with open('scaled_final.pickle', "rb") as f: 
    scaled_features= pickle.load(f)

with open('kmeans8_final.pickle', "rb") as f: 
    kmeans8= pickle.load(f)
    
with open('scaler_final.pickle', "rb") as f: 
    scaler= pickle.load(f)
    

In [34]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

In [35]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [36]:
def Recommender():
    name = input('Enter a song: ')
    track = sp.search(q=name, type='track',limit=3)
    n_songs = len(track['tracks']['items'])
    for i in range(n_songs):
        artist = track['tracks']['items'][i]['artists'][0]['name']
        album = track ['tracks']['items'][i]['album']['name']
        print(i ,'Artist:',artist, '\n  Album:',album)
    select = int(input('Chose a Song: '))
    id_ = track['tracks']['items'][select]['id']
    x = sp.audio_features(id_)
    x =pd.DataFrame(x)[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","duration_ms"]]
    scaled_x = scaler.transform(x)
    cluster = kmeans8.predict(scaled_x)[0]
    recommendation = scaled_features[scaled_features['cluster']==cluster].sample(1)
    recommendation_id = recommendation['id'].iloc[0]
    
    display(play_song(id_))
    display(play_song(recommendation_id))
    
    
Recommender()

Enter a song: no woman no cry
0 Artist: Bob Marley & The Wailers 
  Album: Live!
1 Artist: Bob Marley & The Wailers 
  Album: Natty Dread
2 Artist: Fugees 
  Album: The Score (Expanded Edition)
Chose a Song: 0


/Users/adamkoziak/opt/anaconda3/envs/DA_Env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
